In [18]:
# init
from os import path
import numpy as np
from pynput import keyboard
from pedalboard import Pedalboard, Chorus, Reverb
from pedalboard.io import AudioFile
import music21
from keyboard_layout import KeyboardLayout
from tone import Tone
from audio_player import AudioPlayer
from time_going import Timeline

smplRate = 44100

board = Pedalboard([Chorus(), Reverb(room_size=0.25)])

keyboard_layout = KeyboardLayout()
keyboard_layout.load('complete left to right')


complete left to right loaded as keyboard layout successfully.


In [46]:
tonemaker_1 = Tone(harmonics = [(2,2),(3,3),(4,4),(5,5),(6,6)], fade_duration = 0.06)

def scaling_1(point):
    return 40 * 2**(point/12)

def scaling_2(point):
    scale = [2,2,1,2,2,2,1]
    result = 1
    for i in range(point):
        result += scale[i%len(scale)]
    return result

def final_scaling(x):
    return scaling_1(scaling_2(x))



In [26]:
timeline = Timeline(IS_LOOPED = False)

In [34]:
audioplayer = AudioPlayer(timeline,board)
audioplayer.start()

In [40]:
keyboard_layout.start(final_scaling, audioplayer, tonemaker_1, timeline)

started...


This process is not trusted! Input event monitoring will not be possible until it is added to accessibility clients.


stoped.


In [52]:
import time
score = music21.corpus.parse('bach/bwv65.2.xml')
# score.show()
key = score.analyze('key')
tonicPitchClass = key.tonic.pitchClass
scoreChordify = score.chordify()
scoreChordsFlat = scoreChordify.flatten()
chords = scoreChordsFlat.getElementsByClass('Chord')
for chord in chords:
    print([(p.midi*2-80) for p in chord.pitches],chord.duration.quarterLength)
    position = audioplayer.position

    for p in chord.pitches:
        fundamental = scaling_1(p.midi*2-80)
        tone = tonemaker_1.make(fundamental)/10
        timeline.add(tone, position)
    time.sleep(chord.duration.quarterLength/2)
    
    # i.key = k


    # rootPitchClass = chord.root().pitchClass
    # diff = rootPitchClass - tonicPitchClass
    # if(diff < 0):
    #     diff += (11+1)
    # print(diff)


# print(s.metadata.all())

[10, 40, 48, 58] 1.0
[34, 40, 48, 58] 1.0
[30, 40, 48, 58] 1.0
[28, 44, 58] 1.0
[30, 44, 54, 62] 1.0
[20, 44, 54, 62] 1.0
[26, 44, 54, 62] 1.0
[24, 40, 54, 64] 1.0
[16, 48, 54, 62] 1.0
[26, 40, 50, 58] 1.0
[16, 40, 48, 54] 3.0
[16, 48, 54, 64] 1.0
[40, 48, 54, 64] 1.0
[38, 48, 56, 68] 1.0
[34, 48, 58, 64] 1.0
[28, 44, 58, 64] 1.0
[30, 44, 54, 62] 1.0
[16, 48, 54, 64] 2.0
[40, 48, 54, 64] 1.0
[34, 48, 58, 64] 1.0
[28, 44, 58, 64] 1.0
[20, 44, 52, 64] 1.0
[30, 44, 54, 62] 1.0
[24, 48, 54, 62] 1.0
[18, 48, 54, 58] 1.0
[20, 44, 54, 58] 1.0
[22, 38, 52, 62] 1.0
[24, 38, 48, 56] 1.0
[10, 40, 48, 58] 2.0
[26, 34, 44, 58] 1.0
[26, 38, 44, 54] 1.0
[24, 40, 48, 54] 1.0
[28, 34, 44, 58] 1.0
[30, 44, 62] 1.0
[28, 38, 44, 62] 1.0
[32, 44, 50, 62] 1.0
[34, 40, 48, 64] 1.0


KeyboardInterrupt: 

In [33]:
# closing audio in case of emergency
audioplayer.stop()

In [24]:
timeline.clear()

In [5]:
timeline.save()

didn't save.


stoped.


In [5]:
timeline.load()

test1 loaded as timeline successfually.
